In [1]:
# importing all libraries needed
import numpy as np
import pandas as pd
import os
import time
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

In [2]:
#Stanting timming process
start = time.time()

In [3]:
# saving the main directory
main_directory = os.getcwd()

In [4]:
#Rreading the name of the subfolder
subfolder_with_path = [f.path for f in os.scandir(main_directory) if f.is_dir()]

In [5]:
subfolder_name = [f.name for f in os.scandir(main_directory) if f.is_dir()]

In [6]:
###############################################################################################
# Importing information from "samples overview", "intensisties template" and "brain allen map"
###############################################################################################

# Changing directory to the "zzz_input_and_reference_files" folder
os.chdir(subfolder_with_path[-1])


# Reading "sample_overview.csv" file
samples_overview = pd.read_csv("sample_overview.csv", 
                               dtype={"MOUSE": "string","SAMPLE": int,
                                      "FILES_HD": int,"FILES_BOX": int,
                                      "HEMI": "string","STAIN": "string",
                                      "TX_GROUP": "string", 
                                      "GENDER":"string", 
                                      "CONTEXT":"string"})

In [7]:
# Reading "intensities_template.csv" file
intensities_template = pd.read_csv("intensities_template.csv",
                                   dtype={"IDPath": "string",
                                          "LabelID": int, 
                                          "Region": "string", 
                                          "LabelAbrv": "string"})

In [8]:
# Reading "brain_allen_map.csv" file
brain_allen_map = pd.read_csv("brain_allen_map.csv", 
                               dtype={"raw": "string","allen_1": "string",
                                      "allen_2": "string","allen_3": "string",
                                      "allen_4": "string","allen_5": "string",
                                      "allen_6": "string","allen_7": "string",
                                      "fine": "string","medium": "string",
                                      "coarse": "string", "all":"string"})

In [ ]:
# Changing directory to the "02Volume_files" folder
os.chdir(subfolder_with_path[2])

#Counting number of files on "02Volume_files" folder
list = os.listdir(subfolder_with_path[2]) 
number_files = len(list)
number_volume_files = number_files

In [9]:
#Using and timing  a loop to process all files and populate the main data-frame

volume_data = [] #This array will contain df as elements

i = 0
while i < number_files:

    ##############################################
    # Processing each file
    ##############################################
    
    file = list[i]
    # Saving sample's number 
    sample = file[5:7]


    # Reading file's data
    header = ["PIXEL_COUNT"]
    pixels_count_volume = pd.read_csv(file, names = header)
    pixels_count_volume.insert(0, column = 'INTENSITY', value = range(0, len(pixels_count_volume))) 

    # Adding activity columns to intensitites template data-frame
    output = intensities_template.merge(pixels_count_volume, how='left', left_on='LabelID', right_on='INTENSITY')
    output = output.drop(columns=['INTENSITY'])
    volume_label_name = 'VOLUME'
    output = output.rename(columns = {'PIXEL_COUNT':volume_label_name})
    output['VOLUME'] = output['VOLUME'].div(1000000).round(6)

    # Adding a column of the sample number
    sample_column = np.empty(len(output))
    sample = int(sample)
    sample_column.fill(sample)
    output['SAMPLE'] = sample_column
    ##########################################################################################################################
    
    # store DataFrame in list
    volume_data.append(output)
    
    i += 1
    
#Joinning all elements of the array in a dataframe
volume_data = pd.concat(volume_data)

In [ ]:
# Changing directory to the "03Volume_in_tissue_files" folder
os.chdir(subfolder_with_path[3])

#Counting number of files on "03Volume_in_tissue_files" folder
list = os.listdir(subfolder_with_path[3]) 
number_files = len(list)
number_volintissue_files = number_files

In [ ]:
#Using and timing  a loop to process all files and populate the main data-frame

volume_intissue_data = [] #This array will contain df as elements

#Staring counters

i = 0 #General loop counter

sal_tdt = 0 # Counter of samples where Saline was used and tdTomato expressed
mdma_tdt = 0 # Counter of samples where MDMA was used and tdTomato expressed
meth_tdt = 0 # Counter of samples where Meth was used and tdTomato expressed

sal_cfos = 0 # Counter of samples where Saline was used and cfos expressed
mdma_cfos = 0 # Counter of samples where MDMA was used and cfos expressed
meth_cfos = 0 # Counter of samples where Meth was used and cfos expressed

#These array will contain sample's names as elements
samples_sal_tdt = [] 
samples_mdma_tdt = [] 
samples_meth_tdt = [] 
samples_sal_cfos = []
samples_mdma_cfos = [] 
samples_meth_cfos = []

    ##############################################
    # Processing each file
    ##############################################

while i < number_files:


    file = list[i]
    # Saving sample's number 
    sample = file[5:7]


    # Reading file's data
    header = ["PIXEL_COUNT"]
    pixels_count_volume = pd.read_csv(file, names = header)
    pixels_count_volume.insert(0, column = 'INTENSITY', value = range(0, len(pixels_count_volume))) 

    # Adding activity columns to intensitites template data-frame
    output = intensities_template.merge(pixels_count_volume, how='left', left_on='LabelID', right_on='INTENSITY')
    output = output.drop(columns=['INTENSITY'])
    volume_label_name = 'VOLUME_IN_TISSUE'
    output = output.rename(columns = {'PIXEL_COUNT':volume_label_name})
    output['VOLUME_IN_TISSUE'] = output['VOLUME_IN_TISSUE'].div(1000000).round(6)

    # Adding a column of the sample number
    sample_column = np.empty(len(output))
    sample = int(sample)
    sample_column.fill(sample)
    output['SAMPLE'] = sample_column
    
    # Merging activity and sample overview data-frames
    output= pd.merge(output,samples_overview[['SAMPLE','GENDER',
                                             'hemisphere','marker',
                                             'TX_GROUP','CONTEXT']],on='SAMPLE',how='inner')
    
    #Adding a column with the sample's nickname

    if (output['TX_GROUP'].iloc[0] == 'Saline') and (output['marker'].iloc[0] == 'tdtomato'):
        if output['SAMPLE'].iloc[0] not in samples_sal_tdt:
            sal_tdt += 1
        samples_sal_tdt.append(output['SAMPLE'].iloc[0])
        nickname = 'vSAL_' + str(sal_tdt)
        
    elif (output['TX_GROUP'].iloc[0] == 'MDMA') and (output['marker'].iloc[0] == 'tdtomato'):
        if output['SAMPLE'].iloc[0] not in samples_mdma_tdt:
            mdma_tdt += 1 
        samples_mdma_tdt.append(output['SAMPLE'].iloc[0])
        nickname = 'vMDMA_' + str(mdma_tdt)
        
    elif (output['TX_GROUP'].iloc[0] == 'METH') and (output['marker'].iloc[0] == 'tdtomato'):
        if output['SAMPLE'].iloc[0] not in samples_meth_tdt:
            meth_tdt += 1 
        samples_meth_tdt.append(output['SAMPLE'].iloc[0])
        nickname = 'vMETH_' + str(meth_tdt)
        
    elif (output['TX_GROUP'].iloc[0] == 'Saline') and (output['marker'].iloc[0] == 'cfos'):
        if output['SAMPLE'].iloc[0] not in samples_sal_cfos:
            sal_cfos += 1 
        samples_sal_cfos.append(output['SAMPLE'].iloc[0])
        nickname = 'vSAL_' + str(sal_cfos)
        
    elif (output['TX_GROUP'].iloc[0] == 'MDMA') and (output['marker'].iloc[0] == 'cfos'):
        if output['SAMPLE'].iloc[0] not in samples_mdma_cfos:
            mdma_cfos += 1 
        samples_mdma_cfos.append(output['SAMPLE'].iloc[0])
        nickname = 'vMDMA_' + str(mdma_cfos)
        
    elif (output['TX_GROUP'].iloc[0] == 'METH') and (output['marker'].iloc[0] == 'cfos'):
        if output['SAMPLE'].iloc[0] not in samples_meth_cfos:
            meth_cfos += 1 
        samples_meth_cfos.append(output['SAMPLE'].iloc[0])
        nickname = 'vMETH_' + str(meth_cfos)

    nickname_column = np.empty(len(output))    
    output['nickname'] = nickname_column
    output['nickname'] = nickname
    
    ##########################################################################################################################
    
    # store DataFrame in list
    volume_intissue_data.append(output)
    
    i += 1
    
#Joinning all elements of the array in a dataframeo
volume_intissue_data = pd.concat(volume_intissue_data)

In [ ]:
#filtering volumes in tissue that are les than 50% of the volume in the atlas
volumes_data = pd.merge(volume_intissue_data,volume_data[['VOLUME','SAMPLE']],on='SAMPLE',how='outer')
volumes_data['volumes_ratio'] = volumes_data['VOLUME_IN_TISSUE'] / volumes_data['VOLUME'] 
volumes_data.loc[(volumes_data['volumes_ratio'] <= 0.5),'VOLUME_IN_TISSUE'] = 0
volumes_data.loc[(volumes_data['volumes_ratio'] > 1.0),'VOLUME_IN_TISSUE'] = 0

#Pivoting dataframe to have nicknames as columns
filtered_volumes_data = pd.pivot_table(volumes_data, values='VOLUME_IN_TISSUE', 
                                       index=['raw'],columns=['nickname'], aggfunc=np.sum)

In [ ]:
# Changing directory to the "01Activity_files" folder
os.chdir(subfolder_with_path[1])

#Counting number of files on "01Activity_files" folder
list = os.listdir(subfolder_with_path[1]) 
number_files = len(list)
number_activity_files = number_files

In [10]:
#Using and timing  a loop to process all files and populate the main data-frame

activity_data = [] #This array will contain df as elements

#Staring counters

i = 0 #General loop counter

sal_tdt = 0 # Counter of samples where Saline was used and tdTomato expressed
mdma_tdt = 0 # Counter of samples where MDMA was used and tdTomato expressed
meth_tdt = 0 # Counter of samples where Meth was used and tdTomato expressed

sal_cfos = 0 # Counter of samples where Saline was used and cfos expressed
mdma_cfos = 0 # Counter of samples where MDMA was used and cfos expressed
meth_cfos = 0 # Counter of samples where Meth was used and cfos expressed

#These array will contain sample's names as elements
samples_sal_tdt = [] 
samples_mdma_tdt = [] 
samples_meth_tdt = [] 
samples_sal_cfos = []
samples_mdma_cfos = [] 
samples_meth_cfos = []


    ##############################################
    # Innitiates loop to process each file
    ##############################################


while i < number_files:


    file = list[i]
    # Saving sample's number
    sample = file[5:7]
    
    # Saving rater's name
    rater = file[8:10]


    # Reading file's data
    regions_count_activity_fracc = pd.read_csv(file,low_memory=False)
    regions_count_activity_fracc = regions_count_activity_fracc[regions_count_activity_fracc.PIXEL_COUNT != "PIXEL_COUNT"]

    # Adding-up regions' fracctions
    regions_fracc_1 = regions_count_activity_fracc[["INTENSITY_1", "INTENSITY_1_PERC"]]
    regions_fracc_2 = regions_count_activity_fracc[["INTENSITY_2", "INTENSITY_2_PERC"]]
    regions_fracc_3 = regions_count_activity_fracc[["INTENSITY_3", "INTENSITY_3_PERC"]]
    regions_fracc_1 = regions_fracc_1.rename(columns={"INTENSITY_1": "INTENSITY","INTENSITY_1_PERC" : "COUNTS"})
    regions_fracc_2 = regions_fracc_2.rename(columns={"INTENSITY_2": "INTENSITY","INTENSITY_2_PERC" : "COUNTS"})
    regions_fracc_3 = regions_fracc_3.rename(columns={"INTENSITY_3": "INTENSITY","INTENSITY_3_PERC" : "COUNTS"})
    total_region_activity = regions_fracc_1.append(regions_fracc_2, 
                                                   ignore_index=True).append(regions_fracc_3, ignore_index=True)
    total_region_activity["COUNTS"] = total_region_activity["COUNTS"].astype(float)
    total_region_activity = total_region_activity.groupby(['INTENSITY']).agg('sum').reset_index()
    total_region_activity["COUNTS"] = total_region_activity["COUNTS"].round(0)
    total_region_activity["INTENSITY"] = total_region_activity["INTENSITY"].astype(int)

    # Adding activity columns to intensitites template data-frame
    output = intensities_template.merge(total_region_activity, how='left', left_on='LabelID', right_on='INTENSITY')
    output = output.drop(columns=['INTENSITY'])
    activity_label_name = 'ACTIVITY'
    output = output.rename(columns = {'COUNTS':activity_label_name})

    # Adding a column of the sample number
    sample_column = np.empty(len(output))
    sample = int(sample)
    sample_column.fill(sample)
    output['SAMPLE'] = sample_column
    
    # Adding a column of the rater's name
    rater_column = np.empty(len(output))
    output['rater'] = rater_column
    output['rater'] = rater
    
    # Merging activity and sample overview data-frames
    output= pd.merge(output,samples_overview[['SAMPLE','GENDER',
                                             'hemisphere','marker',
                                             'TX_GROUP','CONTEXT']],on='SAMPLE',how='inner')
    
    #Adding a column with the sample's nickname

    if (output['TX_GROUP'].iloc[0] == 'Saline') and (output['marker'].iloc[0] == 'tdtomato'):
        if output['SAMPLE'].iloc[0] not in samples_sal_tdt:
            sal_tdt += 1
        samples_sal_tdt.append(output['SAMPLE'].iloc[0])
        nickname = 'SAL_' + str(sal_tdt)
        
    elif (output['TX_GROUP'].iloc[0] == 'MDMA') and (output['marker'].iloc[0] == 'tdtomato'):
        if output['SAMPLE'].iloc[0] not in samples_mdma_tdt:
            mdma_tdt += 1 
        samples_mdma_tdt.append(output['SAMPLE'].iloc[0])
        nickname = 'MDMA_' + str(mdma_tdt)
        
    elif (output['TX_GROUP'].iloc[0] == 'METH') and (output['marker'].iloc[0] == 'tdtomato'):
        if output['SAMPLE'].iloc[0] not in samples_meth_tdt:
            meth_tdt += 1 
        samples_meth_tdt.append(output['SAMPLE'].iloc[0])
        nickname = 'METH_' + str(meth_tdt)
        
    elif (output['TX_GROUP'].iloc[0] == 'Saline') and (output['marker'].iloc[0] == 'cfos'):
        if output['SAMPLE'].iloc[0] not in samples_sal_cfos:
            sal_cfos += 1 
        samples_sal_cfos.append(output['SAMPLE'].iloc[0])
        nickname = 'SAL_' + str(sal_cfos)
        
    elif (output['TX_GROUP'].iloc[0] == 'MDMA') and (output['marker'].iloc[0] == 'cfos'):
        if output['SAMPLE'].iloc[0] not in samples_mdma_cfos:
            mdma_cfos += 1 
        samples_mdma_cfos.append(output['SAMPLE'].iloc[0])
        nickname = 'MDMA_' + str(mdma_cfos)
        
    elif (output['TX_GROUP'].iloc[0] == 'METH') and (output['marker'].iloc[0] == 'cfos'):
        if output['SAMPLE'].iloc[0] not in samples_meth_cfos:
            meth_cfos += 1 
        samples_meth_cfos.append(output['SAMPLE'].iloc[0])
        nickname = 'METH_' + str(meth_cfos)

    nickname_column = np.empty(len(output))    
    output['nickname'] = nickname_column
    output['nickname'] = nickname

    
    ##########################################################################################################################
    
    # store DataFrame in list
    activity_data.append(output)
    
    i += 1
    
# see pd.concat documentation for more info
activity_data = pd.concat(activity_data)

#Pivoting dataframe to have nicknames as columns
activity_data = pd.pivot_table(activity_data, values='ACTIVITY', 
                               index=['raw','hemisphere','marker','rater'],columns=['nickname'], aggfunc=np.sum)

#Restarting index to activate'hemisphere','marker'and 'rater' as columns
activity_data = activity_data.reset_index()

In [11]:
activity_data.to_csv('activity_data.csv', index=False)

In [ ]:
# Changing directory to the main path
os.chdir(main_directory)

In [ ]:
#combining dataframes to create the "cell_count_output.csv" file
allen_and_activity = pd.merge(brain_allen_map,activity_data,on='raw',how='right')
cell_count_output = pd.merge(allen_and_activity,filtered_volumes_data,on='raw',how='left')
cell_count_output.to_csv('cell_count_output.csv', index=False)

In [ ]:
#Ending timming process
end = time.time()

# Printing how many files were processed and how much time the process took
files_processed = number_activity_files = number_files+ number_volume_files + number_volintissue_files
print(files_processed,' Files processed ')
print(' ')
print(' Execution time:', round((end - start),2), 'seconds') 